In [1]:
import rasterio
import numpy as np
import pandas as pd

from clasificacion_humedales.utils.utils_maia import directories, read_clustering, save_clustering
from clasificacion_humedales.utils.compare import belonging_dataframe_absolute_values, amount_of_pixels_in_each_cluster, percentage_of_each_cluster

## Directorios

In [2]:
PATH_IN, PATH_OUT = directories()

dir_gt = PATH_OUT + 'clustering_gt/'
dir_out = PATH_OUT + 'side_by_side/'

fn_dates = PATH_OUT + 'clustering/clustering_dates.tif'
fn_pca = PATH_OUT +'clustering/clustering_pca.tif'
fn_gt = dir_gt + 'mapa_gt_clustered.tif'

## Leer datos

In [3]:
gt = read_clustering(fn_gt)
clustering_dates = read_clustering(fn_dates)
clustering_pca = read_clustering(fn_pca)

In [4]:
mask = (gt != -3000)

cgt = gt[mask].flatten()
cpc = clustering_pca[mask].flatten()
cdt = clustering_dates[mask].flatten()

height = gt.shape[0]
width = gt.shape[1]

assert((cdt < 0).sum() == (cgt<0).sum() == (cpc<0).sum() == 0)

## Método principal

A cada clase le asigna el cluster al que pertenece mayoritariamente, ajustado por la cantidad de píxels que tiene

In [5]:
def best_mapping_between(clustering, gt, consider_weigths=True):
    '''busca la mejor asignación de clusters a las clases 
    de manera de maximizar el accuracy de la clasificación
    dados un ground-truth (gt) y un clustering.
    devuelve una serie de pandas que asigna clusters a clases.
    '''
    
    classes_gt = set(gt)
    classes_clustering = set(clustering)
    
    M = belonging_dataframe_absolute_values(clustering, classes_clustering, 'clustering', gt, classes_gt, 'gt')
    M_weigth = M
    if consider_weigths:
        weigths = pd.Series(amount_of_pixels_in_each_cluster(gt))
        M_weigth = M.div(weigths)
    return M_weigth.idxmax(axis=1)

## Vamos lado y lado con los clusters de mejor BIC

### Dates

Encuentro la mejor asignación y la aplico

In [6]:
dic_dt = best_mapping_between(cdt, cgt, consider_weigths=True)
classif_dates = np.array([dic_dt[c] for c in cdt])

In [7]:
dic_dt.to_dict()

{0: 9,
 1: 3,
 2: 22,
 3: 10,
 4: 22,
 5: 3,
 6: 2,
 7: 1,
 8: 25,
 9: 22,
 10: 5,
 11: 1,
 12: 12,
 13: 3,
 14: 20,
 15: 2,
 16: 3,
 17: 4,
 18: 8,
 19: 23,
 20: 15,
 21: 1}

In [8]:
classes_classif_dates=set(classif_dates)

In [9]:
print("Transformando el clustering en clasificación")
print(f'obtuve: {len(classes_classif_dates):>2d} clases: {classes_classif_dates}')

Transformando el clustering en clasificación
obtuve: 14 clases: {1, 2, 3, 4, 5, 8, 9, 10, 12, 15, 20, 22, 23, 25}


Ahora hago para el otro lado

In [10]:
dic_i_dt = best_mapping_between(cgt, classif_dates, consider_weigths=True)
dic_i_dt[-3000]=-3000

In [11]:
gt_filtered_dates = [dic_i_dt[c] for c in cgt]
S = set(gt_filtered_dates)
print("Pegando clases del GT")
print(f'obtuve: {len(S):>2d} Clases: {S}')

Pegando clases del GT
obtuve: 12 Clases: {1, 2, 3, 4, 5, 8, 9, 10, 12, 15, 20, 22}


In [12]:
dic_i_dt.to_frame('vuelta').rename_axis('clase GT').sort_values(['vuelta', 'clase GT'])

,vuelta
clase GT,
-3000,-3000
1,1
13,1
16,1
2,2
3,3
4,4
5,5
18,5


### PCA

Encuentro la mejor asignación y la aplico

In [13]:
dic_pc = best_mapping_between(cpc, cgt, consider_weigths=True)
classif_pca = np.array([dic_pc[c] for c in cpc])

In [14]:
classes_classif_pca = set(classif_pca)
classes_classif_pca.discard(-1)
print(f'Obtuve: {len(classes_classif_pca):>2d} clases {classes_classif_pca}')

Obtuve: 17 clases {0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 13, 14, 15, 16, 20, 21, 22}


Ahora hago para el otro lado

In [15]:
dic_i_pc = best_mapping_between(cgt, classif_pca, consider_weigths=True)
dic_i_pc[-3000]=-3000

In [16]:
gt_filtered_pca = [dic_i_pc[c] for c in cgt]
S = set(gt_filtered_pca)
print("Pegando clases del GT:")
print(f'Obtuve: {len(S):>2d} | Clases: {S}')

Pegando clases del GT:
Obtuve: 17 | Clases: {0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 13, 14, 15, 16, 20, 21, 22}


In [17]:
dic_i_pc.to_frame('vuelta').rename_axis('clase GT').sort_values(['vuelta', 'clase GT'])

,vuelta
clase GT,
-3000,-3000
0,0
24,0
1,1
2,2
3,3
18,4
5,5
10,5


## Guardo en geotiff

Guardo en geotiff la clasificación de Dates

In [23]:
image = gt.copy()
image[mask] = classif_dates

save_clustering(fn_gt, dir_out + 'classif_dates_weigths.tif', image)

Guardo en geotiff el GT tamizado por Dates

In [24]:
image = gt.copy()
image[mask] = gt_filtered_dates
    
save_clustering(fn_gt, dir_out + 'gt_filtered_dates_weigths.tif', image)

Guardo en geotiff la clasificación de PCA

In [25]:
image = gt.copy()
image[mask] = classif_pca
    
save_clustering(fn_gt, dir_out + 'classif_pca_weigths.tif', image)

Guardo en geotiff el GT tamizado por PCA

In [26]:
image = gt.copy()
image[mask] = gt_filtered_pca
    
save_clustering(fn_gt, dir_out + 'gt_filtered_pca_weigths.tif', image)